In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/viktor/mlfs-book-a-version
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
"""project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']version

today = datetime.date.today()

location_str"""

'project = hopsworks.login()\nfs = project.get_feature_store() \nsecrets = hopsworks.get_secrets_api()\n\n# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks\nAQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value\nlocation_str = secrets.get_secret("SENSOR_LOCATION_JSON").value\nlocation = json.loads(location_str)\n\ncountry=location[\'country\']\ncity=location[\'city\']\nstreet=location[\'street\']\naqicn_url=location[\'aqicn_url\']\nlatitude=location[\'latitude\']\nlongitude=location[\'longitude\']version\n\ntoday = datetime.date.today()\n\nlocation_str'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
"""air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)"""

"air_quality_fg = fs.get_feature_group(\n    name='air_quality',\n    version=1,\n)\nweather_fg = fs.get_feature_group(\n    name='weather',\n    version=1,\n)"

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

"""aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df"""

'aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)\naq_today_df'

In [6]:
#aq_today_df.info()

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
"""hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df"""

"hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)\nhourly_df = hourly_df.set_index('date')\n\n# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast\n# We only want the daily weather data, so only get weather at 12:00\ndaily_df = hourly_df.between_time('11:59', '12:01')\ndaily_df = daily_df.reset_index()\ndaily_df['date'] = pd.to_datetime(daily_df['date']).dt.date\ndaily_df['date'] = pd.to_datetime(daily_df['date'])\ndaily_df['city'] = city\ndaily_df"

In [8]:
#daily_df.info()

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
#air_quality_fg.insert(aq_today_df)

In [10]:
# Insert new data
#weather_fg.insert(daily_df, wait=True)

In [11]:
#Add to the new version with lagg (kept all things above as is, but just append this for the C-part):

"""air_quality_fg_V2 = fs.get_feature_group(
    name='air_quality',
    version=2,
)"""

"air_quality_fg_V2 = fs.get_feature_group(\n    name='air_quality',\n    version=2,\n)"

In [12]:
"""aq_today_df_lagg = aq_today_df.copy()
today_date = aq_today_df["date"].dt.date.iloc[0]

#Get the rows that match the location
history = air_quality_fg_V2.read()

history = history.query("country == @country and city == @city and street == @street") 

#print("Rows before filtering:", len(history))
history = history[history["date"].dt.date < today_date]
#print("Rows after filtering:", len(history))

sorted_history = history.sort_values("date")

last_days = sorted_history.tail(3)
last_pm25_values = last_days["pm25"].tolist()
aq_today_df_lagg['pm25_lag1'] = last_pm25_values[2]
aq_today_df_lagg['pm25_lag2'] = last_pm25_values[1]
aq_today_df_lagg['pm25_lag3'] = last_pm25_values[0]

for col in ["pm25", "pm25_lag1", "pm25_lag2", "pm25_lag3"]:  #got an error that said it was double without this!
    aq_today_df_lagg[col] = aq_today_df_lagg[col].astype("float32")

aq_today_df_lagg"""

'aq_today_df_lagg = aq_today_df.copy()\ntoday_date = aq_today_df["date"].dt.date.iloc[0]\n\n#Get the rows that match the location\nhistory = air_quality_fg_V2.read()\n\nhistory = history.query("country == @country and city == @city and street == @street") \n\n#print("Rows before filtering:", len(history))\nhistory = history[history["date"].dt.date < today_date]\n#print("Rows after filtering:", len(history))\n\nsorted_history = history.sort_values("date")\n\nlast_days = sorted_history.tail(3)\nlast_pm25_values = last_days["pm25"].tolist()\naq_today_df_lagg[\'pm25_lag1\'] = last_pm25_values[2]\naq_today_df_lagg[\'pm25_lag2\'] = last_pm25_values[1]\naq_today_df_lagg[\'pm25_lag3\'] = last_pm25_values[0]\n\nfor col in ["pm25", "pm25_lag1", "pm25_lag2", "pm25_lag3"]:  #got an error that said it was double without this!\n    aq_today_df_lagg[col] = aq_today_df_lagg[col].astype("float32")\n\naq_today_df_lagg'

In [13]:
#air_quality_fg_V2.insert(aq_today_df_lagg)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store


In [14]:
#load sensor info. 
config_path=f"{root_dir}/config/sensors.json"


with open(config_path, "r") as f:
    SENSORS = json.load(f)

print("Loaded sensors:", len(SENSORS))
SENSORS


Loaded sensors: 6


[{'country': 'uk',
  'city': 'dundee',
  'street': 'lochee road',
  'aqicn_url': 'https://api.waqi.info/feed/@5971/',
  'csv_file': 'lochee road'},
 {'country': 'uk',
  'city': 'dundee',
  'street': 'whitehall street',
  'aqicn_url': 'https://api.waqi.info/feed/@9579/',
  'csv_file': 'whitehall street'},
 {'country': 'uk',
  'city': 'dundee',
  'street': 'meadowside',
  'aqicn_url': 'https://api.waqi.info/feed/@5973/',
  'csv_file': 'meadowside'},
 {'country': 'uk',
  'city': 'dundee',
  'street': 'seagate',
  'aqicn_url': 'https://api.waqi.info/feed/@5974/',
  'csv_file': 'seagate'},
 {'country': 'uk',
  'city': 'dundee',
  'street': 'mains loan',
  'aqicn_url': 'https://api.waqi.info/feed/@5972/',
  'csv_file': 'mains loan'},
 {'country': 'uk',
  'city': 'dundee',
  'street': 'broughty ferry road',
  'aqicn_url': 'https://api.waqi.info/feed/@5970/',
  'csv_file': 'broughty ferry road'}]

In [15]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()
today = datetime.date.today()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value



2025-11-19 02:38:28,296 INFO: Initializing external client
2025-11-19 02:38:28,297 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-19 02:38:29,707 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296538


In [16]:
def ingest_daily(country, city, street, aqicn_url):
    print("Ingesting PM25 values for; ", street)

    air_quality_fg_V2 = fs.get_feature_group(
        name='air_quality',
        version=10,
    )
    #Retrieve Today's Air Quality data (PM2.5) from the AQI API
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    print(aq_today_df)
    print(aq_today_df.info())

    aq_today_df_lagg = aq_today_df.copy()
    
    today_date = aq_today_df["date"].dt.date.iloc[0]
    
    #Get the rows that match the location
    history = air_quality_fg_V2.read()
    history = history.query("country == @country and city == @city and street == @street") 
    
    #print("Rows before filtering:", len(history))
    history = history[history["date"].dt.date < today_date]
    #print("Rows after filtering:", len(history))
    sorted_history = history.sort_values("date")
    
    last_days = sorted_history.tail(3)
    last_pm25_values = last_days["pm25"].tolist()
    aq_today_df_lagg['pm25_lag1'] = last_pm25_values[2]
    aq_today_df_lagg['pm25_lag2'] = last_pm25_values[1]
    aq_today_df_lagg['pm25_lag3'] = last_pm25_values[0]
    
    for col in ["pm25", "pm25_lag1", "pm25_lag2", "pm25_lag3"]:  #got an error that said it was double without this!
        aq_today_df_lagg[col] = aq_today_df_lagg[col].astype("float32")
    
    print(aq_today_df_lagg)
    
    air_quality_fg_V2.insert(aq_today_df_lagg)

In [17]:
latitude = None
longitude = None
for sensor in SENSORS:
    street = sensor["street"]
    #Load the location!
    street_safe = street.replace(" ", "_").upper()
    secret_name = f"SENSOR_LOCATION_JSON_{street_safe}"
    location_str = secrets.get_secret(secret_name).value #location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
    location = json.loads(location_str)
    country=location['country']
    city=location['city']
    street=location['street']
    aqicn_url=location['aqicn_url']
    latitude=location['latitude']
    longitude=location['longitude']

    print(location_str)
    
    ingest_daily(country, city, street, aqicn_url)
    
    
    
    

{"country": "uk", "city": "dundee", "street": "lochee road", "aqicn_url": "https://api.waqi.info/feed/@5971/", "latitude": 56.46, "longitude": -2.97}
Ingesting PM25 values for;  lochee road
   pm25 country    city       street       date  \
0   7.0      uk  dundee  lochee road 2025-11-19   

                                 url  
0  https://api.waqi.info/feed/@5971/  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes
None
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.31s) 
  

Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_10_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296538/jobs/named/air_quality_10_offline_fg_materialization/executions
{"country": "uk", "city": "dundee", "street": "whitehall street", "aqicn_url": "https://api.waqi.info/feed/@9579/", "latitude": 56.46, "longitude": -2.97}
Ingesting PM25 values for;  whitehall street
   pm25 country    city            street       date  \
0   8.0      uk  dundee  whitehall street 2025-11-19   

                                 url  
0  https://api.waqi.info/feed/@9579/  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non

Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


{"country": "uk", "city": "dundee", "street": "meadowside", "aqicn_url": "https://api.waqi.info/feed/@5973/", "latitude": 56.46, "longitude": -2.97}
Ingesting PM25 values for;  meadowside
   pm25 country    city      street       date  \
0   5.0      uk  dundee  meadowside 2025-11-19   

                                 url  
0  https://api.waqi.info/feed/@5973/  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes
None
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.46s) 
   pm2

...

Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


{"country": "uk", "city": "dundee", "street": "mains loan", "aqicn_url": "https://api.waqi.info/feed/@5972/", "latitude": 56.46, "longitude": -2.97}
Ingesting PM25 values for;  mains loan
   pm25 country    city      street       date  \
0   3.0      uk  dundee  mains loan 2025-11-19   

                                 url  
0  https://api.waqi.info/feed/@5972/  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes
None
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.31s) 
   pm2

Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


{"country": "uk", "city": "dundee", "street": "broughty ferry road", "aqicn_url": "https://api.waqi.info/feed/@5970/", "latitude": 56.46, "longitude": -2.97}
Ingesting PM25 values for;  broughty ferry road
   pm25 country    city               street       date  \
0   4.0      uk  dundee  broughty ferry road 2025-11-19   

                                 url  
0  https://api.waqi.info/feed/@5970/  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes
None
Finished: Reading data from Hopsworks, using Hopsworks F

Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


In [18]:
#wether data:

weather_fg = fs.get_feature_group(
    name='weather',
    version=10,
)


hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
print(daily_df)
print(daily_df.info())
weather_fg.insert(daily_df, wait=True)

Coordinates 56.5°N -2.75°E
Elevation 15.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
        date  temperature_2m_mean  precipitation_sum  wind_speed_10m_max  \
0 2025-11-19                 3.05                0.0           22.104116   
1 2025-11-20                 4.35                0.0           14.512064   
2 2025-11-21                 1.55                0.0           11.440978   
3 2025-11-22                 7.00                0.0           13.896187   
4 2025-11-23                 7.05                0.1           10.948973   
5 2025-11-24                 6.95                0.4           25.407431   
6 2025-11-25                 4.65                0.5           12.413476   
7 2025-11-26                 3.35                0.0            9.000000   
8 2025-11-27                 7.90                0.5           25.704100   

   wind_direction_10m_dominant    city  
0                   307.056610  dundee  
1                   316.004974  dundee  
2               

Uploading Dataframe: 100.00% |██████████████████████████████████| Rows 9/9 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_10_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296538/jobs/named/weather_10_offline_fg_materialization/executions
2025-11-19 02:40:11,024 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-19 02:40:14,210 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-19 02:41:55,783 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-19 02:41:55,932 INFO: Waiting for log aggregation to finish.
2025-11-19 02:42:31,098 INFO: Execution finished successfully.


(Job('weather_10_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 759134
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 9,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-19T01:39:53.000821Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column